In [1]:
pip install fosforml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 31.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 81.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 65.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 47.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 84.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━

In [2]:
pip show fosforml

Name: fosforml
Version: 1.1.9
Summary: A Python package for registering machine learning models directly to the Snowflake Model Registry, leveraging Snowflake ML capabilities.
Home-page: https://gitlab.fosfor.com/fosfor-decision-cloud/intelligence/refract-sdk.git
Author: Mahesh Gadipea
Author-email: mahesh.gadipea@fosfor.com
License: 
Location: /tmp/pip_packages
Requires: cloudpickle, scikit-learn, snowflake-ml-python
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

In [2]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import RandomForestRegressor
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [3]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [4]:
# import pandas as pd
# df = pd.read_csv("/notebooks/notebooks/Data/HR_CHURN.csv")
# snow_df = session.create_dataframe(df)
# input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])

In [4]:
import pandas as pd
df = pd.read_csv("/notebooks/notebooks/HR_CHURN_LATEST.csv")
snow_df = session.create_dataframe(df)
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])

In [5]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']
input_data_frame.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECTS"  |"AVERAGE_MONTHLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT_COMPANY"  |"PROMOTION_LAST_5YEARS"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0             |0.38                  |0.53               |2                  |157                      |3                     |0                |1               |0                        |
|2.0       |7.0           |1             |0.8                   |0.86               |5                  |262                      |6    

In [6]:
pipeline = RandomForestRegressor(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)

The version of package 'snowflake-snowpark-python' in the local environment is 1.26.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [7]:
pred_snow_df = pipeline.predict(apply_label_encoding(test_df))

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [8]:
from fosforml import register_model

In [9]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_1_3_10",
  description="This is a test model",
  flavour="snowflake",
    dataset_source="HR_CHURN",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)

Calculating build time metrics

Progress: ██████████████████                                                     25.0%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.*'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-snowpark-python' in the local environment is 1.26.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.


Calculating build time metrics

Progress: ███████████████████████████████████                                    50.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████                   75.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_6FE87D1A_0347_44B8_8A78_A94759C49908_FDC_HR_CHURN_1_3_10' registered successfully."

In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_2_3_10",
  description="This is a test model",
  flavour="snowflake",
    dataset_source="HR_CHURN",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)

In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_3_3_10",
  description="This is a test model",
  flavour="snowflake",
    dataset_source="HR_CHURN",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)

In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_4_3_10",
  description="This is a test model",
  flavour="snowflake",
    dataset_source="HR_CHURN",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)

In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_5_3_10",
  description="This is a test model",
  flavour="snowflake",
    dataset_source="HR_CHURN",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="regression",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)